In [1]:
import cv2 as cv
import numpy as np
# cap = cv.VideoCapture(0)
# ret, first_frame = cap.read()
# if not ret:
#     print("Failed to capture frame.")
#     return
# first_frame = cv.flip(first_frame, 1)

In [4]:


def calibration():
    cap = cv.VideoCapture(0)
    ret, first_frame = cap.read()
    if not ret:
        print("Failed to capture frame.")
        return
    first_frame = cv.flip(first_frame, 1)
    width = first_frame.shape[1]
    height = first_frame.shape[0]
    w = 70
    h = 70
    joints = {"shoulder": (width // 2 - w // 2, height // 5, w, h), # (x, y, w, h) - xy is the top left corner
              "elbow": (int(width // 2 + 1 * w), height // 5, w, h),
              "wrist": (int(width // 2 + 2 * w), height // 5, w, h),
              "hip": (width // 2 - w // 2, int(height // 5 + 2.75 * h), w, h),
              "knee": (width // 2 - w // 2, int(height // 5 + 4.5 * h), w, h),
              "ankle": (width // 2 - w // 2, int(height // 5 + 5.5 * h + 50), w, h)}
   
    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture frame.")
            break
        frame = cv.flip(frame, 1)
        for _, rect in joints.items():
            x, y, w, h = rect
            cv.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv.imshow("Frame", frame)
        key = cv.waitKey(1) # When user is ready (fits his joints in the boxes), press space
        if key == 32: # Space ASCII code
            break
    cap.release()
    cv.waitKey(1)
    cv.destroyAllWindows()
    cv.waitKey(1)
    centers = {joint: (rect[0] + rect[2] // 2, rect[1] + rect[3] // 2) for joint, rect in joints.items()}
    return centers

In [1]:
import numpy as np
import cv2 as cv

# Example calibration function; replace with your actual function
def calibration():
    return {'joint1': (100, 150), 'joint2': (200, 250)}

JOINTS_LOCATIONS = calibration()  # Returns a dictionary with the {joint: joint_coords}

feature_params = dict(maxCorners=100,
                      qualityLevel=0.3,
                      minDistance=7,
                      blockSize=7)

# Parameters for lucas kanade optical flow
lk_params = {
    "winSize": (15, 15),
    "maxLevel": 2,
    "criteria": (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03)
}

# Create some random colors
color = np.random.randint(0, 255, (100, 3))

# Initialize video capture (webcam)
cap = cv.VideoCapture(0)
if not cap.isOpened():
    print("Error: Cannot open the webcam.")
    exit()

ret, old_frame = cap.read()
if not ret:
    print("Failed to capture frame.")
    cap.release()
    exit()

joint_coords = list(JOINTS_LOCATIONS.values())
old_gray = cv.cvtColor(old_frame, cv.COLOR_BGR2GRAY)

p0 = np.array(joint_coords, dtype=np.float32)
mask = np.zeros_like(old_frame)

while True:
    ret, frame = cap.read()
    if not ret:
        print("ERROR: Failed To Capture Frame")
        break

    frame = cv.flip(frame, 1)
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # Calculate optical flow
    p1, st, err = cv.calcOpticalFlowPyrLK(old_gray, frame_gray, p0, None, **lk_params)

    if p1 is not None:
        st = st.flatten()
        good_new = p1[st == 1]
        good_old = p0[st == 1]
    else:
        continue

    # Draw the tracks
    for i, (new, old) in enumerate(zip(good_new, good_old)):
        a, b = new.ravel()
        c, d = old.ravel()
        a, b, c, d = int(a), int(b), int(c), int(d)
        cv.circle(frame, (a, b), 5, color[i].tolist(), -1)

    img = cv.add(frame, mask)
    cv.imshow('frame', img)

    k = cv.waitKey(1)
    if k == 27 or k == ord('q'):
        break

    # Update the previous frame and previous points
    old_gray = frame_gray.copy()
    p0 = good_new.reshape(-1, 1, 2)

cap.release()
cv.destroyAllWindows()


AttributeError: module 'cv2' has no attribute 'TERM_CRITERIA_EPS'

In [3]:
from gtts import gTTS
import pygame
import io
#pip install gtts pygame

def text_to_speech(text, language='en', slow=False):
    # Convert text to speech
    tts = gTTS(text=text, lang=language, slow=slow)
    
    # Save the audio to a BytesIO object instead of a file
    audio_data = io.BytesIO()
    tts.write_to_fp(audio_data)
    audio_data.seek(0)
    
    # Initialize pygame mixer
    pygame.mixer.init()
    
    # Load the audio data into pygame
    pygame.mixer.music.load(audio_data, 'mp3')
    
    # Play the audio
    pygame.mixer.music.play()
    
    # Keep the program running until the audio is done playing
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)

# Example usage
english_text = "Firstly, raise your hands, then bend forward with your back straight, all the way to your shin"
text_to_speech(english_text)


In [7]:
import cv2 as cv
import tkinter as tk
from PIL import Image, ImageTk

# Function to update the video frames in real time
def update_frame():
    ret, frame = cap.read()  # Capture real-time video frame
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)  # Convert to RGB format
        img = Image.fromarray(frame)  # Convert to PIL Image
        imgtk = ImageTk.PhotoImage(image=img)  # Convert to ImageTk format
        label.config(image=imgtk)  # Update label with new frame
        label.image = imgtk

    # Overlay the pre-recorded video on top
    ret2, overlay_frame = overlay_cap.read()
    if ret2:
        overlay_frame = cv2.resize(overlay_frame, (overlay_width, overlay_height))  # Resize the overlay video frame
        overlay_frame = cv2.cvtColor(overlay_frame, cv2.COLOR_BGR2RGB)
        overlay_img = Image.fromarray(overlay_frame)
        overlay_imgtk = ImageTk.PhotoImage(image=overlay_img)
        overlay_label.config(image=overlay_imgtk)
        overlay_label.image = overlay_imgtk

    label.after(10, update_frame)  # Update the frame every 10 ms

# Initialize Tkinter window
root = tk.Tk()
root.title("Real-Time Video with Overlay")

# Initialize the real-time video capture (webcam)
cap = cv2.VideoCapture(0)

# Initialize the overlay video capture (pre-recorded video)
overlay_video_path = r'C:\Users\adva4\Downloads\כללי\לימודים\עיבוד תמונה ספרתי\project\example_adva_full.mp4'
overlay_cap = cv2.VideoCapture(overlay_video_path)

# Create a label to display the real-time video
label = tk.Label(root)
label.pack(fill=tk.BOTH, expand=True)

# Set the size of the overlay video box
overlay_width = 200  # Width of the small video
overlay_height = 150  # Height of the small video

# Create a smaller label for the overlay video
overlay_label = tk.Label(root)
overlay_label.place(x=10, y=10)  # Position the overlay in the top-left corner

# Start updating frames
update_frame()

# Start the Tkinter event loop
root.mainloop()

# Release the video captures when done
cap.release()
overlay_cap.release()
cv2.destroyAllWindows()


NameError: name 'cv2' is not defined